<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> 
        <h1>Natural Language Processing</h1>
        <h1>Sentiment Analysis</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
        @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import string
import gzip
from collections import Counter

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

import watermark

%matplotlib inline
%load_ext watermark

List out the versions of all loaded libraries

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 9c8c00758f3a2fa8e55e08f5aad405a157ca5dd2

pandas    : 2.2.3
watermark : 2.4.3
nltk      : 3.8.1
numpy     : 1.26.4
matplotlib: 3.8.0



Set the default style

In [3]:
plt.style.use('./d4sci.mplstyle')

## Word counting

We start by taking the simplest approach and simply counting positive and negative words. We'll use Hu and Liu's Lexicon from their 2004 KDD paper: https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html

In [4]:
pos = np.loadtxt('data/positive-words.txt', dtype='str', comments=';')
neg = np.loadtxt('data/negative-words.txt', dtype='str', comments=';')

In [5]:
pos

array(['a+', 'abound', 'abounds', ..., 'zenith', 'zest', 'zippy'],
      dtype='<U20')

In [6]:
neg

array(['2-faced', '2-faces', 'abnormal', ..., 'zealous', 'zealously',
       'zombie'], dtype='<U24')

Create a dictionary and assign the valence to each positive and negative word

In [7]:
valence = {}

for word in pos:
    valence[word.lower()] = +1
    
for word in neg:
    valence[word.lower()] = -1

That now we can use to define a sentiment measuring function that returns the valence of a sentence or piece of text. Notice that we use the valence directly from the dictionary instead of treating positive and negative words separatly. This will prove useful later on ;)

In [8]:
def sentiment(words, valence):    
    word_count = 0
    score = 0
    
    for word in words:
        if word in valence:
            score += valence[word]
            word_count += 1
    
    return score/word_count

Now let's test our simple code with some simple examples

In [9]:
texts = ["I'm very happy",
         "The product is pretty annoying, and I hate it",
         "I'm sad",
        ]

for text in texts:
    words = nltk.word_tokenize(text.lower())
    print(text, ':', sentiment(words, valence))

I'm very happy : 1.0
The product is pretty annoying, and I hate it : -0.3333333333333333
I'm sad : -1.0


This is a bit surprising. One might expect the second sentence to be negative, after all "pretty annoying" and "hate" sound pretty negative. However, since each word in taken by itself, regardless of context we end up with:

In [10]:
words = nltk.word_tokenize(texts[1].lower())

for word in words:
    if word in valence:
        print(word, valence[word])

pretty 1
annoying -1
hate -1


We'll see in a bit how to handle cases like this, but the solution requires two important changes to our current approach: modifier words and real valued weights

## Continuous weights

VADER is a state of the art sentiment analysis tool. Here we will use their excelent and well documented [lexicon](https://github.com/cjhutto/vaderSentiment) to explore non binary weights. Their approach is significantly more advanced than what we present here, but some of the fundamental ideas are the same

In [11]:
vader = pd.read_csv("data/vader_lexicon.txt", sep='\t', header=None, names=['word', 'mean', 'std', 'scores'])

The vader lexicon includes a lot of interesting information:

In [12]:
vader.iloc[np.random.randint(0, vader.shape[0], 10)]

,word,mean,std,scores
3488,grouch,-2.2,0.87178,"[-2, -3, -3, -3, -2, -1, -3, -1, -1, -3]"
7491,yearning,0.5,1.02470,"[0, 1, 0, 1, 0, 3, 0, 1, -1, 0]"
75,1337,2.1,1.13578,"[3, 1, 4, 0, 2, 3, 1, 2, 2, 3]"
2234,distractibility,-1.3,1.10000,"[-1, -1, -3, -1, 0, -3, -2, 0, -2, 0]"
4154,irritableness,-1.7,0.64031,"[-2, -2, -2, -1, -2, -1, -1, -3, -1, -2]"
7046,unstable,-1.5,0.50000,"[-2, -2, -1, -2, -1, -1, -1, -2, -2, -1]"
331,nitl,-1.5,0.92195,"[-1, -1, -2, -3, -1, -3, -1, -2, 0, -1]"
5596,represses,-1.3,1.00499,"[-1, 0, -3, -1, 0, -3, -1, -1, -2, -1]"
4442,lousiness,-1.7,0.64031,"[-2, -2, -1, -1, -2, -2, -3, -2, -1, -1]"
4223,joyfulness,2.7,1.00499,"[4, 3, 1, 3, 3, 3, 4, 2, 1, 3]"


Smilies are also included and, in addition to the average sentiment of each word (in column 1) and it's standard deviation (in column 2) it provides the raw human generated scores in column 3. So that we may easily check (and possibly modify) their weights. To extract the raw scores for the word "love" we could simply do:

In [13]:
print(vader.iloc[4446])

word                                love
mean                                 3.2
std                                  0.4
scores    [3, 3, 3, 3, 3, 3, 3, 4, 4, 3]
Name: 4446, dtype: object


From where we can also extract the individual scores

In [14]:
scores = eval(vader.iloc[4446][3])
print(scores)

[3, 3, 3, 3, 3, 3, 3, 4, 4, 3]


/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_32593/3485897463.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  scores = eval(vader.iloc[4446][3])


In [15]:
scores[8]

4

And we can see that 8/10 people thought that the word love should receive a score of 3 and two others a score of 4. This gives us insight into how uniform the scores are.  If for some reason, we thought that there was some problem with the 2 values of 4 or perhaps just not appropriate to our purposes we might discard them and recalculate the valence of the word. 

One justification for this might be the fact that the scores for the closely related word, "loved", are significantly different with a wider range of variation in the human scores

In [16]:
vader.iloc[4447]

word                               loved
mean                                 2.9
std                                  0.7
scores    [3, 3, 4, 2, 2, 4, 3, 2, 3, 3]
Name: 4447, dtype: object

Now we convert this dataset into a dictionary similar to the one we used above

In [17]:
valence_vader = dict(vader[['word', 'mean']].values)

In [18]:
valence_vader['loved']

2.9

To use this new dictionary we just have to modify the arguments to the sentiment_modified function:

In [19]:
sentiment(nltk.word_tokenize("It was not not very very good"), valence_vader)

1.9

One important detail to keep in mind is that scores obtained through different methods are not comparable. In this example, the score of the sentence "It was not not very very good" went from 2.25 to 4.27 when we switched dictionaries. This is due not only to different levels of coverage in differnet dictionaries but also to differnet choices in the possible ranges of values.

# NLTK Sentiment Analysis

nltk provides us with a VADER implementation that we can easily access

In [20]:
sentiment = SentimentIntensityAnalyzer()

And we can even check that it is using the same file as we have been using

In [21]:
print(sentiment.lexicon_file[:1000])

$:	-1.5	0.80623	[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]
%)	-0.4	1.0198	[-1, 0, -1, 0, 0, -2, -1, 2, -1, 0]
%-)	-1.5	1.43178	[-2, 0, -2, -2, -1, 2, -2, -3, -2, -3]
&-:	-0.4	1.42829	[-3, -1, 0, 0, -1, -1, -1, 2, -1, 2]
&:	-0.7	0.64031	[0, -1, -1, -1, 1, -1, -1, -1, -1, -1]
( '}{' )	1.6	0.66332	[1, 2, 2, 1, 1, 2, 2, 1, 3, 1]
(%	-0.9	0.9434	[0, 0, 1, -1, -1, -1, -2, -2, -1, -2]
('-:	2.2	1.16619	[4, 1, 4, 3, 1, 2, 3, 1, 2, 1]
(':	2.3	0.9	[1, 3, 3, 2, 2, 4, 2, 3, 1, 2]
((-:	2.1	0.53852	[2, 2, 2, 1, 2, 3, 2, 2, 3, 2]
(*	1.1	1.13578	[2, 1, 1, -1, 1, 2, 2, -1, 2, 2]
(-%	-0.7	1.26886	[-1, 2, 0, -1, -1, -2, 0, 0, -3, -1]
(-*	1.3	1.26886	[4, 1, 2, 0, 2, -1, 1, 2, 1, 1]
(-:	1.6	0.8	[2, 2, 1, 3, 1, 1, 1, 3, 1, 1]
(-:0	2.8	0.87178	[3, 2, 3, 4, 3, 2, 3, 1, 4, 3]
(-:<	-0.4	2.15407	[-3, 3, -1, -1, 2, -1, -2, 3, -3, -1]
(-:o	1.5	0.67082	[3, 1, 1, 2, 2, 2, 1, 1, 1, 1]
(-:O	1.5	0.67082	[3, 1, 1, 2, 2, 2, 1, 1, 1, 1]
(-:{	-0.1	1.57797	[-2, -3, 1, -2, 1, 1, 0, 0, 2, 1]
(-:|>*	1.9	0.83066	


The sentiment analyzer expects a full sentence as input and returns scores along various dimensions:

In [22]:
sentiment.polarity_scores("good")

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4404}

The scores for negative, neutral and positive all add up to one, while compound can range from -1 to +1.

## Modifiers

Our approach so far has ignored the possibility of modifiers. The first step towards improving our approach is to define a dictionary of modifiers

In [23]:
modifiers = {
    "very": 1.5,
    "much": 1.3,
    "not": -1,
    "pretty": 1.5,
    "somewhat": 1.2
}

And to change our sentiment measuring function to take the modifiers into account.

In [24]:
def sentiment_modified(text, valence, modifiers, verbose=False):
    words = nltk.word_tokenize(text.lower())
    
    word_count = 0
    score = 0
    ngrams = [[]]
    
    # generate ngrams
    for i in range(len(words)):
        word = words[i]
        
        if word in modifiers:
            ngrams[-1].append(word)
            continue

        if word in valence:
            ngrams[-1].append(word)
        else:
            if len(ngrams[-1]) > 0:
                ngrams.append([])

    score = 0
    
    # Remove the trailing empty ngram if necessary
    if len(ngrams[-1]) == 0:
        ngrams = ngrams[:-1]

    for ngram in ngrams:
        value = 1

        for word in ngram:
            if word in modifiers:
                value *= modifiers[word]
            elif word in valence:
                value *= valence[word]

        if verbose:
            print(ngram, value)

        score += value

    return score/len(ngrams)

This implementation is still relatively simple, but, as you can see, the results are already better.

In [25]:
print(texts[1])

The product is pretty annoying, and I hate it


In [26]:
sentiment_modified(texts[1], valence, modifiers, True)

['pretty', 'annoying'] -1.5
['hate'] -1


-1.25

A more complete implementation would be more careful in handling the modifiers and would build larger ngrams so that cases like this one would also work:

In [27]:
sentiment_modified("It was not very good", valence, modifiers, True)

['not', 'very', 'good'] -1.5


-1.5

And even more complex (and unrealistic) examples work fine

In [28]:
sentiment_modified("It was not not very very good", valence, modifiers, True)

['not', 'not', 'very', 'very', 'good'] 2.25


2.25

Fortunately for us, the nltk vader implementation correctly handles modifiers under the hood

In [29]:
sentiment.polarity_scores("good")

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4404}

In [30]:
sentiment.polarity_scores("not good")

{'neg': 0.706, 'neu': 0.294, 'pos': 0.0, 'compound': -0.3412}

In [31]:
sentiment.polarity_scores("very good")

{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'compound': 0.4927}

In [32]:
sentiment.polarity_scores("not very good")

{'neg': 0.567, 'neu': 0.433, 'pos': 0.0, 'compound': -0.3865}

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>